# Tensor的基本操作

## 1. 新建Tensor

In [1]:
import torch
import numpy as np

### 1.1 根据给定的shape直接新建tensor
1. ones, zeros, empty等操作和numpy相似
2. shape可以是tuple，也可以直接给各个dimension的大小，而不package到tuple中
3. 用manual_seed()来设定随机种子

In [15]:
shape = (2, 3)
zeros = torch.zeros(shape)
i = torch.ones((2, 3), dtype=torch.int16) # 指定数据类型

print(zeros, '\n', i)
torch.is_storage(zeros)

tensor([[0., 0., 0.],
        [0., 0., 0.]]) 
 tensor([[1, 1, 1],
        [1, 1, 1]], dtype=torch.int16)


False

In [3]:
## random initialization
# random seed
torch.manual_seed(1729)
r1 = torch.rand(shape)   # [0, 1)均匀分布
print(r1)

r2 = torch.rand(shape)   # [0, 1)均匀分布
print(r2)

torch.manual_seed(1729)
r3 = torch.rand(shape)   # [0, 1)均匀分布
print(r3 == r1)

tensor([[0.3126, 0.3791, 0.3087],
        [0.0736, 0.4216, 0.0691]])
tensor([[0.2332, 0.4047, 0.2162],
        [0.9927, 0.4128, 0.5938]])
tensor([[True, True, True],
        [True, True, True]])


### 1.2 将python list，tuple或numpy array转变成tensor类型

In [4]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)
print(x_data, x_data.dtype)

tensor([[1, 2],
        [3, 4]]) torch.int64


In [5]:
## 将np_array转变成tensor
np_array = np.array(data)
x_np = torch.tensor(np_array)
y_np = torch.from_numpy(np_array)
print(x_np, '\n', y_np)

tensor([[1, 2],
        [3, 4]]) 
 tensor([[1, 2],
        [3, 4]])


In [6]:
## 可以识别混合类型
data = [[1, 2],(3.5, 4)]
x_data = torch.tensor(data)
print(x_data, x_data.dtype)

tensor([[1.0000, 2.0000],
        [3.5000, 4.0000]]) torch.float32


In [7]:
## tensor转变成numpy array
pytorch_rand = torch.rand(2, 3)
print(pytorch_rand)

numpy_rand = pytorch_rand.numpy()
print(numpy_rand)

tensor([[0.2332, 0.4047, 0.2162],
        [0.9927, 0.4128, 0.5938]])
[[0.23321933 0.40465623 0.2162376 ]
 [0.99269456 0.41275233 0.59382254]]


### 1.3 用已有tensor的shape和datatype来新建tensor
这里要注意数据类型变化

In [8]:
x_ones = torch.ones_like(x_data)
print(f'ones tensor with shape as x_data:', x_ones)

# x_rand = torch.rand_like(x_data) # 错，x_data原来的value是int，要override
x_rand = torch.rand_like(x_data, dtype=torch.float)
print(x_rand)

ones tensor with shape as x_data: tensor([[1., 1.],
        [1., 1.]])
tensor([[0.6128, 0.1519],
        [0.0453, 0.5035]])


## 2. tensor的属性

### 2.1 数据类型
新建tensor的默认类型是float32

In [9]:
# 设置数据类型的两种方式
a = torch.ones((2, 3), dtype=torch.int16)
print(a)

c = a.to(torch.float32)
print(c)

tensor([[1, 1, 1],
        [1, 1, 1]], dtype=torch.int16)
tensor([[1., 1., 1.],
        [1., 1., 1.]])


### 2.2 存储位置
1. 默认情况下，tensor是在cpu上创建的，可以用tensor.to()将tensor转移到gpu上，或者直接在gpu上新建tensor
2. 一个expression中的运算对象要在同一个device上，cpu和gpu上的数据无法在一个expression中处理
3. cpu上的tensor可以跟numpy共享底层的memory location，但gpu上的tensor不行。因为gpu上只能放tensor，不能放numpy数据类型
4. 要注意，在cpu和gpu之间做数据的迁移很耗费时间。

In [10]:
## 在gpu上新建tensor
x = torch.rand(2, 2, device='cuda')
print(x)

tensor([[0.3344, 0.2640],
        [0.2119, 0.0582]], device='cuda:0')


In [11]:
## 将现有tensor移动到gpu
x = torch.ones(2, 3)
if torch.cuda.is_available():
    y = x.to('cuda')
print(x.shape, x.dtype, x.device)
print(y.shape, y.dtype, y.device)

# mul = tensor * tensor2 # 错，他们一个在cpu上，一个在gpu上

torch.Size([2, 3]) torch.float32 cpu
torch.Size([2, 3]) torch.float32 cuda:0


In [12]:
## cpu上的tensor可以跟numpy共享memory location。改变其中一个的值，另一个也变
t = torch.ones(5)
n = t.numpy()
print("改变前：", t, n)
t.add_(2)
print("改变tensor，numpy对象也变：", t, n)
np.add(n, 2, out=n)
print("改变numpy对象后，tensor也变：", t, n)

改变前： tensor([1., 1., 1., 1., 1.]) [1. 1. 1. 1. 1.]
改变tensor，numpy对象也变： tensor([3., 3., 3., 3., 3.]) [3. 3. 3. 3. 3.]
改变numpy对象后，tensor也变： tensor([5., 5., 5., 5., 5.]) [5. 5. 5. 5. 5.]


In [13]:
## 但在gpu上无法做numpy类型对象的计算
if torch.cuda.is_available():
    t = t.to('cuda')
#     n = n.to('cuda') # numpy对象不能移到gpu上，因为没有.to method
print(t)
# n = t.numpy() # 报错：can't convert cuda:0 device type tensor to numpy.

tensor([5., 5., 5., 5., 5.], device='cuda:0')
